## Load Tile Placements Table

In [ ]:
import sqlite3
conn = sqlite3.connect('../dataset/tile_placements.db')
c = conn.cursor()

In [ ]:
conn.close()

---

## Create Tile Placements Table

In [ ]:
c.execute('''CREATE TABLE tile_placements 
            (timestamp INTEGER, user_hash TEXT, user_name TEXT, x_coord INTEGER, y_coord INTEGER, color INTEGER)''')
conn.commit()

---

## Insert CSV File to Tile Placements Table 

In [ ]:
def insert_tile_placement(timestamp, user_hash, user_name, x_coord, y_coord, color):
    c.execute('''INSERT INTO tile_placements (timestamp, user_hash, user_name, x_coord, y_coord, color)
                 VALUES (?, ?, ?, ?, ?, ?);''', (timestamp, user_hash, user_name, x_coord, y_coord, color))

In [ ]:
read_data = open("../../ShortcutRedditDir/RawData/place/tile_placements_with_usernames.csv").readlines()[1:]
for line in read_data:
    line = line.split(',')
    insert_tile_placement(int(line[0])/1000, line[1], line[2],  int(line[3]), int(line[4]), int(line[5]))
conn.commit()

---

## Empty Table

In [ ]:
c.execute("DELETE FROM tile_placements;")
conn.commit()

---

##  Show Size of Table 

In [ ]:
c.execute('SELECT COUNT(*) FROM tile_placements;')
print (c.fetchone())

---

##  Show Contents of Table 

In [ ]:
c.execute('SELECT * FROM tile_placements;')
print (c.fetchone())

---

##  Generate Index for Table 

In [ ]:
c.execute('CREATE INDEX idx_placement_username ON tile_placements (user_name)')

In [ ]:
c.execute('CREATE INDEX idx_placement_coord ON tile_placements (x_coord, y_coord)')

In [ ]:
c.execute('CREATE INDEX idx_placement_timestamp ON tile_placements (x_coord, y_coord, timestamp)')

---

##  Search Contents of Table 

In [ ]:
%%time
time = 1491238734
c.execute("""SELECT t1.*
FROM tile_placements t1
WHERE t1.timestamp = (SELECT MAX(t2.timestamp)
                FROM tile_placements t2
                WHERE t2.x_coord = t1.x_coord AND t2.y_coord = t1.y_coord AND t2.timestamp <= ?)""", (time,))
rows = c.fetchall()
print(len(rows))

---

In [ ]:
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np
directory = "/home/srivbane/shared/caringbridge/data/projects/place-project/data/"
df = pd.read_feather(directory+"derived/feather/user.ft")
df.head()

In [ ]:
class PixelHistory:
    def __init__(self, time_color_map, timestamp_list):
        self.time_color_map = time_color_map
        self.timestamp_list = np.array(timestamp_list)
        
    def get_color_at_time(self, ts):
        ts = self.timestamp_list[self.timestamp_list <= ts][-1]
        color = self.time_color_map[ts]
        return color
        
class TilePlacements:
    def __init__(self):
        self.d = {}
        
    def build_from_df(self, df):
        for key, group in tqdm(df.groupby(by=['x_coordinate', 'y_coordinate'], sort=False)):
            x, y = key
            time_color_map = {ts: color for ts, color in zip(group.timestamp, group.color)}
            timestamp_list = group.timestamp
            timestamp_list.sort_values()
            self.d[key] = PixelHistory(time_color_map, timestamp_list)
    
    def get_colors_at_timestamp(self, ts):
        colors = np.array((1000, 1000))
        for x in range(1000):
            for y in range(1000):
                colors[x, y] = self.d[(x,y)].get_color_at_time(ts)

In [ ]:
tp = TilePlacements()
tp.build_from_df(df)

##  Get Unique UserNames in Table 

In [ ]:
%%time
c.execute("SELECT DISTINCT user_name FROM tile_placements;")
print (c.fetchall())